In [29]:
import math


unimodal_functions = {
    '-5x^5 + 4x^4 - 12x^3 + 11x^2 - 2x + 1, [-0.5,0.5]': (lambda x: -5*x**5 + 4*x**4 - 12*x**3 + 11*x**2 - 2*x + 1, (-0.5,0.5)),
    'lg(x-2)^2 + lg(10-x)^2 - x^0.2, [6.9,9]': (lambda x: math.log10(x-2)**2 + math.log10(10-x)**2 - math.pow(x,0.2), (6,9.9)),
    '-3x*sin(0.75x) + e^(-2x), [0,2pi]': (lambda x: -3*x*math.sin(0.75*x) + math.exp(-2*x), (0,2*math.pi)),
    'e^(3x) + 5e^(-2x), [0,1]': (lambda x: math.exp(3*x) + 5*math.exp(-2*x), (0,1)),
    '0.2x*lg(x) + (x-2.3)^2, [0.5,2.5]': (lambda x: 0.2*x*math.log10(x) + (x - 2.3)**2, (0.5,2.5))
}



In [30]:


def dichotomy_method(function,start,end,epsilon=1e-5):
    delta = epsilon/4
    function_calls_number = 0
    step_details = []

    while end - start > epsilon:
        center = (start + end)/2
        a = function(center - delta)
        b = function(center + delta)
        function_calls_number += 2
        if a > b:
            start = center - delta
        elif a < b:
            end = center + delta
        else:
            start = center - delta
            end = center + delta
        step_details.append((start, end, a if a < b else b))
    return ((start + end)/2, function_calls_number, step_details)



In [31]:


def golden_ratio_method(function,start,end,epsilon=1e-5):
    function_calls_number = 0
    step_details = []
    phi = (3 - math.sqrt(5))/2

    x1 = start + (end - start)*phi
    x2 = end - (end - start)*phi

    x1_value = function(x1)
    x2_value = function(x2)
    function_calls_number += 2

    while end - start > epsilon:
        if x1_value > x2_value:
            start = x1
            x1 = x2
            x2 = end + start - x1
            x1_value = x2_value
            x2_value = function(x2)
        else:
            end = x2
            x2 = x1
            x1 = start + end - x2
            x2_value = x1_value
            x1_value = function(x1)
        function_calls_number += 1
        step_details.append((start, end, x1_value if x1_value < x2_value else x2_value))
    return ((x1 + x2)/2, function_calls_number, step_details)



In [32]:


def get_n_fibonacci(number):
    fs = [1, 1]
    for i in range(1,number):
        fs.append(fs[i] + fs[i-1])
    return fs


def get_fibonacci_less(number):
    fs = [1, 1]
    i = 1
    while fs[i] < number:
        fs.append(fs[i] + fs[i-1])
        i += 1
    return fs


def fibonacci_method(function,start,end,n=0,epsilon=1e-5):
    function_calls_number = 0
    step_details = []
    if n < 1:
        fibonacci_sequence = get_fibonacci_less((end-start)/epsilon)
        n = len(fibonacci_sequence)
    else:
        fibonacci_sequence = get_n_fibonacci(n)
    
    x1 = start + (end - start)*fibonacci_sequence[n-3]/fibonacci_sequence[n-1]
    x2 = start + (end - start)*fibonacci_sequence[n-2]/fibonacci_sequence[n-1]

    x1_value = function(x1)
    x2_value = function(x2)
    function_calls_number += 2

    while n > 2:
        if x1_value > x2_value:
            start = x1
            x1 = x2
            x2 = end - (x1 - start)
            x1_value = x2_value
            x2_value = function(x2)
        else:
            end = x2
            x2 = x1
            x1 = start + end - x2
            x2_value = x1_value
            x1_value = function(x1)
        step_details.append((start, end, x1_value if x1_value < x2_value else x2_value))
        function_calls_number += 1
        n -= 1
    return ((x1 + x2)/2, function_calls_number, step_details)




In [33]:


def quadratic_method(function,start,end,epsilon=1e-5):
    function_calls_number = 0
    step_details = []
    pivot = (start + end)/2

    start_value = function(start)
    pivot_value = function(pivot)
    end_value = function(end)

    function_calls_number += 3
    
    pivot_step = (end + start) / 100
    if start_value < pivot_value:
        pivot_step = -pivot_step
        u_prev = end
    else:
        u_prev = start
    while start_value < pivot_value or end_value < pivot_value:
        pivot += pivot_step
        pivot_value = function(pivot)
        function_calls_number += 1
 
    u = pivot - (((pivot - start) ** 2) * (pivot_value - end_value) - ((pivot - end) ** 2) * (pivot_value - start_value)) / 2 / (
            (pivot - start) * (pivot_value - end_value) - (pivot - end) * (pivot_value - start_value))

    while abs(u - u_prev) > epsilon:
        u_value = function(u)
        if u <= pivot and u_value > pivot_value:
            start = u
            start_value = u_value
        elif u > pivot and u_value > pivot_value:
            end = u
            end_value = u_value
        elif u > pivot and u_value <= pivot_value:
            start = pivot
            start_value = pivot_value
            pivot = u
            pivot_value = u_value
        elif u <= pivot and u_value <= pivot_value:
            end = pivot
            end_value = pivot_value
            pivot = u
            pivot_value = u_value
        u_prev = u
        u = pivot - (((pivot - start) ** 2) * (pivot_value - end_value) - ((pivot - end) ** 2) * (pivot_value - start_value)) / 2 / (
            (pivot - start) * (pivot_value - end_value) - (pivot - end) * (pivot_value - start_value))
        step_details.append((start, end, u_value))
        function_calls_number += 1
    return (0.5 * (u + u_prev), function_calls_number, step_details)



In [34]:


def sign(number):
    return 1 if number >= 0 else -1


def brent_method(function,start,end,epsilon=1e-5):
    function_calls_number = 0
    step_details = []
    phi = 0.5 * (3 - math.sqrt(5))

    x = (start + end)/2
    omega = x
    v = x

    x_value = function(x)
    omega_value = function(omega)
    v_value = function(v)
    function_calls_number += 3

    d = end - start
    e = end - start
    u = end
    u_prev = start
    while abs(u - u_prev) > epsilon:
        g = e
        e = d
        if len({x, omega, v}) == len({x_value, omega_value, v_value}) == 3:
            segment_values = {x : x_value, v : v_value, omega : omega_value}
            segment = sorted([x, omega, v])
            u_prev = u
            u = segment[1] - 0.5 * (((segment[1] - segment[0]) ** 2) * (segment_values[segment[1]] - segment_values[segment[2]]) - ((segment[1] - segment[2]) ** 2) * (segment_values[segment[1]] - segment_values[segment[0]])) / ((segment[1] - segment[0]) * (segment_values[segment[1]] - segment_values[segment[2]]) - (segment[1] - segment[2]) * (segment_values[segment[1]] - segment_values[segment[0]]))
        if (start+epsilon) < u < (end-epsilon) and abs(u - x) < 0.5 * g:
            d = abs(u - x)
        else:
            u_prev = u
            if x < 0.5 * (end + start):
                u = x + phi*(end - x)
                d = end - x
            else:
                u = x - phi*(x - start)
                d = x - start
        if abs(u - x) < epsilon:
            u_prev = u
            u = x + sign(u-x) * epsilon
        u_value = function(u)
        function_calls_number += 1
        if u_value <= x_value:
            if u >= x:
                start = x
            else:
                end = x
            v = omega
            omega = x
            x = u
            v_value = omega_value
            omega = x_value
            x_value = u_value
        else:
            if u >= x:
                end = u
            else:
                start = u
            if u_value <= omega_value or omega == x:
                v = omega
                omega = u
                v_value = omega_value
                omega_value = u_value
            elif u_value <= v_value or omega == v:
                v = u
                v_value = u_value
        step_details.append((start, end, u_value))
    return (0.5 * (u + u_prev), function_calls_number, step_details)
              


In [35]:
import pandas


method_names = [
    'dichotomy',
    'golden ratio',
    'fibonacci',
    'quadratic',
    'brent'
]

table_columns = [
    'x', 'function calls', 'iterations'
]


for readable_func in unimodal_functions:
    func = unimodal_functions[readable_func][0]
    start_range = unimodal_functions[readable_func][1][0]
    end_range = unimodal_functions[readable_func][1][1]

    d_x, d_func_calls, d_steps_info = dichotomy_method(func,start_range,end_range)
    g_x, g_func_calls, g_steps_info = golden_ratio_method(func,start_range,end_range)
    f_x, f_func_calls, f_steps_info = fibonacci_method(func,start_range,end_range)
    q_x, q_func_calls, q_steps_info = quadratic_method(func,start_range,end_range)
    b_x, b_func_calls, b_steps_info = brent_method(func,start_range,end_range)

    data = []
    data.append([d_x, d_func_calls, len(d_steps_info)])
    data.append([g_x, g_func_calls, len(g_steps_info)])
    data.append([f_x, f_func_calls, len(f_steps_info)])
    data.append([q_x, q_func_calls, len(q_steps_info)])
    data.append([b_x, b_func_calls, len(b_steps_info)])

    print(readable_func)
    print(pandas.DataFrame(data,method_names,table_columns))
    print()



-5x^5 + 4x^4 - 12x^3 + 11x^2 - 2x + 1, [-0.5,0.5]
                     x  function calls  iterations
dichotomy     0.109861              36          18
golden ratio  0.109858              26          24
fibonacci     0.109866              26          24
quadratic     0.109864              12           9
brent         0.110874              11           8

lg(x-2)^2 + lg(10-x)^2 - x^0.2, [6.9,9]
                     x  function calls  iterations
dichotomy     8.726906              40          20
golden ratio  8.726907              29          27
fibonacci     8.726907              29          27
quadratic     8.726781             116         113
brent         8.694839               6           3

-3x*sin(0.75x) + e^(-2x), [0,2pi]
                     x  function calls  iterations
dichotomy     2.706474              42          21
golden ratio  2.706476              30          28
fibonacci     2.706469              30          28
quadratic     2.706474              11           8
brent  